In [1]:
from encoder.audio import preprocess_wav
from encoder.inference import embed_utterance, load_model as load_embedding_model
from pathlib import Path
import soundfile as sf


from synthesizer.inference import Synthesizer


from vocoder.inference import load_model as load_vocoder , infer_waveform

from synthesizer.audio import inv_mel_spectrogram
from synthesizer.hparams import hparams
import numpy as np
import os
import torch
from joblib import Parallel, delayed
import re

In [9]:
encoder_model_path = Path("models/encoder.pt")
syn_model_path = Path("models/seab_trained/synthesizer.pt")
vocoder_model_path = Path("models/vocoder.pt")

In [39]:




#target_spk_path = "demo/source/p374_113.wav"

In [4]:
load_embedding_model(encoder_model_path)
load_vocoder(vocoder_model_path)
syn = Synthesizer(syn_model_path)

Loaded encoder "encoder.pt" trained to step 1564501
Building Wave-RNN
Trainable Parameters: 4.481M
Loading model weights at models/vocoder.pt
Synthesizer using device: cpu


In [194]:
#wav = preprocess_wav(source_spk_path)
#source_spk_embedding = embed_utterance(wav)

wav = preprocess_wav(target_spk_path)
target_spk_embedding = embed_utterance(wav)

In [195]:
syn_output = syn.synthesize_spectrograms(texts = ["This cake is great. It's so delicious and moist."],embeddings = target_spk_embedding)[0]

Trainable Parameters: 30.870M
Loaded synthesizer "synthesizer.pt" trained to step 297000

| Generating 1/1


Done.



In [196]:
#mel_gan_vocoder =  torch.hub.load('descriptinc/melgan-neurips', 'load_melgan')

In [197]:
#syn_output = np.load("models/001/mel-spectrograms/mel-prediction-step-1000_sample_1.npy").T

In [198]:
#wav_out = inv_mel_spectrogram(syn_output,hparams= hparams)
wav_out= infer_waveform(syn_output)
#wav_out = mel_gan_vocoder.inverse(torch.tensor(syn_output).unsqueeze(0)).numpy()[0]

{| ████████████████ 57000/57600 | Batch Size: 6 | Gen Rate: 2.1kHz | }

In [199]:
wav_out.shape

(45200,)

In [200]:
sf.write("./demo/generated/{}".format(os.path.split(target_spk_path)[-1]),wav_out,16000)

In [ ]:
# VCTK p376 p374 p364 p363 p362 p360 p340 p298 p310 p345

In [10]:
def infer(source_spks,source_dir,dump_dir,target_speechs):

    args = []
    for source_spk in source_spks:
        for index,target_speech in enumerate(target_speechs):
            source_spk_path  = os.path.join(source_dir,source_spk)
            args.append((source_spk_path,target_speech,index))

  

    print(args)
        

    def makeAndWrite(arg):
        
        load_embedding_model(encoder_model_path)
        load_vocoder(vocoder_model_path,verbose= False)
        syn = Synthesizer(syn_model_path,verbose= False)

        source_spk_path,target_speech,index = arg
        wav = preprocess_wav(source_spk_path)
        source_spk_embedding = embed_utterance(wav)
        syn_output = syn.synthesize_spectrograms(texts = [target_speech],embeddings = source_spk_embedding)[0]

        wav_out= infer_waveform(syn_output)
        dump_spk_dir = os.path.join(dump_dir,os.path.split(source_spk_path)[1].split(".wav")[0])
        if(os.path.exists(dump_spk_dir) == False):
            os.mkdir(dump_spk_dir)
        
        name = re.sub(r'[^a-zA-Z0-9_]', '', "_".join(target_speech.lower().split()))
        dump_path = os.path.join(dump_spk_dir,name+ ".wav")

        sf.write(dump_path,wav_out,16000)


    Parallel(n_jobs=-1)(delayed(makeAndWrite)(arg)
        for arg in args)
    

    #for arg in args:
    #    makeAndWrite(arg)

In [16]:
target_speeches = ["it took me quite a long time to develop a voice, and now that I have it I'm not going to be silent.",
"Be a voice, not an echo.",
"I'm sorry Dave. I'm afraid I can't do that.",
"This cake is great. It's so delicious and moist.",
"Prior to November 22, 1963."]

original_speeches = ["My ideal morning begins with hot coffee"]

spks = ["speaker1.wav", "speaker2.wav", "speaker3.wav", "speaker4.wav", "speaker5.wav"]
spks = ["speaker3.wav"]

In [17]:
infer(source_spks = spks ,source_dir= "demo/source", dump_dir= "demo/generated",target_speechs= original_speeches)

[('demo/source/speaker3.wav', 'My ideal morning begins with hot coffee', 0)]
Loaded encoder "encoder.pt" trained to step 1564501
Trainable Parameters: 4.481M
Trainable Parameters: 30.870M
